In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# 基础路径
base_path = Path(r"C:\Users\田\Desktop\python实操\kaggle\Natural Language Processing with Disaster Tweets")  # 你的原始路径

In [2]:
sample_submission = pd.read_csv(base_path / 'sample_submission.csv')
test = pd.read_csv(base_path / 'test.csv')
train = pd.read_csv(base_path / 'train.csv')

> **环境**: Python 3.13, PyTorch 2.x, Transformers 4.x

<big>sft<big>

In [6]:
%%time
# ==============================
# 灾难推文分类 - SFT风格BERT
# 保持BERT模型，但用生成式训练方式
# ==============================

import pandas as pd
import numpy as np
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,  # 用回分类模型
    get_linear_schedule_with_warmup
)
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

# ==============================
# 1. 配置参数
# ==============================
SEED = 42
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 2
LR = 2e-5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 使用BERT模型
MODEL_NAME = r'C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased'

# ==============================
# 2. 数据处理函数
# ==============================
def clean_text(tweet):
    """清理文本"""
    tweet = str(tweet)
    tweet = re.sub(r'https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

def format_sft_prompt(text, label=None):
    """
    格式化指令提示 - 为BERT调整
    """
    instruction = "判断以下推文是否描述真实灾难事件："
    prompt = f"{instruction} {text}"
    return prompt

# ==============================
# 3. 数据集类
# ==============================
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = format_sft_prompt(str(self.texts[idx]))
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
        }
        
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        else:
            # 测试数据没有标签
            item['labels'] = torch.tensor(0, dtype=torch.long)  # 占位符
            
        return item

# ==============================
# 4. 训练和评估函数
# ==============================
def train_epoch(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0
    
    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        if (batch_idx + 1) % 10 == 0:
            avg_loss = total_loss / (batch_idx + 1)
            print(f"  Batch {batch_idx+1}/{len(dataloader)}, Loss: {avg_loss:.4f}")
    
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader):
    model.eval()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_targets, all_predictions, average='macro')
    return f1, all_predictions

# ==============================
# 5. 主训练流程
# ==============================
def main():
    print("🚀 开始灾难推文分类训练")
    print(f"设备: {DEVICE}")
    print(f"模型: {MODEL_NAME}")
    
    # 加载数据
    print("\n📊 加载和预处理数据...")
    df_train = train
    df_test = test
    
    # 清理文本
    df_train['text_cleaned'] = df_train['text'].apply(clean_text)
    df_test['text_cleaned'] = df_test['text'].apply(clean_text)
    
    # 初始化tokenizer和模型
    print(f"\n🔄 加载tokenizer和模型")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # 关键修复：为BERT设置padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
    # 加载模型
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2
    )
    model.resize_token_embeddings(len(tokenizer))
    model = model.to(DEVICE)
    
    print(f"✅ 模型加载完成")
    
    # 准备数据
    from sklearn.model_selection import train_test_split
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        df_train['text_cleaned'].values,
        df_train['target'].values,
        test_size=0.2,
        random_state=SEED,
        stratify=df_train['target']
    )
    
    # 创建数据集
    train_dataset = TweetDataset(train_texts, train_labels, tokenizer, MAX_LEN)
    val_dataset = TweetDataset(val_texts, val_labels, tokenizer, MAX_LEN)
    test_dataset = TweetDataset(df_test['text_cleaned'].values, None, tokenizer, MAX_LEN)
    
    # 创建数据加载器
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # 优化器
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=int(total_steps * 0.1),
        num_training_steps=total_steps
    )
    
    # 训练循环
    print(f"\n🎯 开始训练，共 {EPOCHS} 个epoch")
    best_f1 = 0
    
    for epoch in range(EPOCHS):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        print(f"{'='*50}")
        
        # 训练
        train_loss = train_epoch(model, train_loader, optimizer, scheduler)
        print(f"训练损失: {train_loss:.4f}")
        
        # 评估
        f1_score_val, _ = evaluate_model(model, val_loader)
        print(f"验证集F1分数: {f1_score_val:.4f}")
        
        if f1_score_val > best_f1:
            best_f1 = f1_score_val
            torch.save(model.state_dict(), f"best_model_epoch{epoch+1}.pth")
            print(f"✅ 保存最佳模型，F1: {best_f1:.4f}")
    
    # 测试集预测
    print(f"\n🔮 在测试集上生成预测...")
    if EPOCHS > 0:
        model.load_state_dict(torch.load(f"best_model_epoch{EPOCHS}.pth"))
    
    model.eval()
    test_predictions = []
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            test_predictions.extend(predictions.cpu().numpy())
    
    # 生成提交文件
    print(f"\n💾 生成提交文件...")
    submission = sample_submission.copy()
    submission['target'] = test_predictions[:len(submission)]
    
    submission.to_csv('bert_sft_submission.csv', index=False)
    print(f"✅ 提交文件已保存: bert_sft_submission.csv")
    print(f"📈 最佳F1分数: {best_f1:.4f}")
    
    return submission

# ==============================
# 6. 运行训练
# ==============================
if __name__ == "__main__":
    try:
        submission = main()
        print("\n✅ 训练完成!")
        
        # 验证前几个预测
        print("\n🔍 验证预测结果:")
        print(f"测试集大小: {len(submission)}")
        print(f"预测分布: 0={sum(submission['target']==0)}, 1={sum(submission['target']==1)}")
        
    except Exception as e:
        print(f"\n❌ 错误: {e}")
        import traceback
        traceback.print_exc()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 开始灾难推文分类训练
设备: cuda
模型: C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased

📊 加载和预处理数据...

🔄 加载tokenizer和模型
✅ 模型加载完成

🎯 开始训练，共 2 个epoch

Epoch 1/2
  Batch 10/191, Loss: 0.6845
  Batch 20/191, Loss: 0.6714
  Batch 30/191, Loss: 0.6396
  Batch 40/191, Loss: 0.6000
  Batch 50/191, Loss: 0.5700
  Batch 60/191, Loss: 0.5487
  Batch 70/191, Loss: 0.5285
  Batch 80/191, Loss: 0.5154
  Batch 90/191, Loss: 0.5154
  Batch 100/191, Loss: 0.5096
  Batch 110/191, Loss: 0.5018
  Batch 120/191, Loss: 0.4936
  Batch 130/191, Loss: 0.4858
  Batch 140/191, Loss: 0.4796
  Batch 150/191, Loss: 0.4766
  Batch 160/191, Loss: 0.4713
  Batch 170/191, Loss: 0.4677
  Batch 180/191, Loss: 0.4664
  Batch 190/191, Loss: 0.4636
训练损失: 0.4643
验证集F1分数: 0.8393
✅ 保存最佳模型，F1: 0.8393

Epoch 2/2
  Batch 10/191, Loss: 0.3331
  Batch 20/191, Loss: 0.3515
  Batch 30/191, Loss: 0.3366
  Batch 40/191, Loss: 0.3315
  Batch 50/191, Loss: 0.3362
  Batch 60/191, Loss: 0.3342
  Batch 70/191, Loss: 0.3372
  Batch 

In [10]:
# 注：在真实业务中，'user_id' 会来自用户行为日志，此处为演示模拟